# Predicting Fat Level Of Cheese Types

## Introduction 

In the following we will try to find the best way to predict fat level of different cheese types using their properties. This is an important question to be answered as customers make their decisions according fat level, and when introducing new products, this factor should be considered before making decisions on milk type, processing, etc., as they all influence the fat level. 

This is a classification problem since the fat level is reported as high and low and not the exact percentage. This means that our attempt will be on recognizing different classes. It is also noteworthy that there are only two classes and this is a binary problem.

The positive class is the higher fat group as recognizing these products is of higher importance, because missing them is undesirable.  


## Exploratory Data Analasys

A variety of packages and functions will be used in this work and they're imported here.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import make_scorer
import scipy
from scipy.stats import randint
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
import altair as alt
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from scipy.stats import lognorm, loguniform, randint

The data is stored in cheese_data.csv and has been obtained from Kaggle and follows an Open Government Licence (Canada). The original data was found on the Government of Canada's Open Government Portal but has unfortunately been taken down. However, the dataused here, has been modified, wrangled and cleaned by UBC extended learning machine learning team.
In the following, the data is read and saved as cheese_df, and is studied briefly.

In [ ]:
cheese_df = pd.read_csv('cheese_data.csv')

cheese_df.info()

In order to getting a better understanding of the features, it is useful to study their unique values. Later, these results will be used to group features and preprocess them and handle missing values.
To keep the result, text columns have been omitted.

In [ ]:
for col in ['ManufacturerProvCode', 'ManufacturingTypeEn', 'MoisturePercent', 'Organic', 'CategoryTypeEn', 'MilkTypeEn', 'MilkTreatmentTypeEn', 'FatLevel']:
    print(col,cheese_df[col].unique(),cheese_df[col].dtype
         )

The FatLevel column which is the target, has categorical values. Since our algorithms work with numbers, we implement this change to have 1 for higher and 0 for lower fat level.

In [ ]:
cheese_df['FatLevel']=cheese_df['FatLevel'].replace({'lower fat': 0, 'higher fat': 1})
cheese_df['FatLevel']

Let's now split the data to train and test subset and then separate our feature vectors from the target. The FatLevel column is used as the target y, and other columns are used as features. This should be done for both train_df and test_df and the results is saved in objects named X_train, y_train, X_test and y_test.

In [ ]:
train_df,test_df = train_test_split(cheese_df,test_size=0.2,random_state=123)
X_train,y_train = train_df.drop(columns=['FatLevel']),train_df['FatLevel']
X_test,y_test = test_df.drop(columns=['FatLevel']),test_df['FatLevel']

Charts are valueable tools to evaluate our features, their overall relation to the target and distributions. 
In the folloing two charts are presented. The first one illustrates the distribution of target values, and in the second figure, the relation between moisture and fat level in different cheese types are shown. 

In [ ]:
target_distribution = alt.Chart(cheese_df).mark_bar().encode(x='FatLevel',y=alt.Y('count()',title='number of samples in each fat level group')).properties(title='distribution of values in target column (FatLevel)')
target_distribution

the target class is not balanced but the positive class is not very rare. To be more precise:

In [ ]:
cheese_df['FatLevel'].value_counts()

In [ ]:
Moisture_Fat= alt.Chart(cheese_df).mark_point().encode(x='MoisturePercent',y='FatLevel')
Moisture_Fat

It can be seen that moisture range in lower fat class, lies in larger amounts and it seems that higher moisture contriutes to lowering the fat level. Some key values can be noteworthy.

In [ ]:
cheese_df.groupby(by='FatLevel')['MoisturePercent'].describe()

In the next step, features are groupped and for each group a transformer is provided to handle missing values, and either implement scaling or OneHotEncoding depending on the nature of features.

In [ ]:
numeric_features = ['MoisturePercent']
categorical_features = ['ManufacturerProvCode','ManufacturingTypeEn','CategoryTypeEn','MilkTypeEn','MilkTreatmentTypeEn']
binary_features = ['Organic']
ordinal_features = []
drop_features = ['CheeseId','RindTypeEn']
text_features = ['FlavourEn','CharacteristicsEn','CheeseName']


In [ ]:
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler())

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(dtype=int, handle_unknown="ignore"))

binary_transformer = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(dtype=int, handle_unknown="error",drop='if_binary'))

text_transformer = make_pipeline(CountVectorizer())



## Methods And Results
In the following we define a column transformer using make_column_transformer called preprocessor for the numerical, categorical, and remainding feature types.


In [ ]:
preprocessor = make_column_transformer(
    (numeric_transformer,numeric_features),
    (categorical_transformer,categorical_features),
    (binary_transformer,binary_features),
    ("drop",drop_features),
    ("drop",text_features))

Using pipelines we can apply our preproccers and classifiers. In this report four models, DummyClassifier as a baseline model, KNeighborsClassifier and SVC as logistic models, and RandomForestClassifier as an ensemble of decisiontrees, are used. 
As it was shown, the target is unbalanced. For now, we continue with the unbalanced data. Later with weightings, we will address this problem and compare results.

In [ ]:
pipe_ub_dum=make_pipeline(preprocessor,DummyClassifier())
pipe_ub_knn=make_pipeline(preprocessor,KNeighborsClassifier())
pipe_ub_svc=make_pipeline(preprocessor,SVC())
pipe_ub_rfc=make_pipeline(preprocessor,RandomForestClassifier(random_state=77))

In the following, a scoring dictionary is provided in order to examine our models with different metrics. This will be used in later steps.

In [ ]:
scoring_meth={'accuracy':'accuracy','precision':'precision','recall':'recall','f1':'f1'}

At this point, using cross-validations on X_train and y_train using our pipelines, we can compare our models' performances. Averages of all results are reported to ease the comparison.  

In [ ]:
scores_ub_dum=pd.DataFrame(cross_validate(pipe_ub_dum,X_train,y_train,cv=5,return_train_score=True,scoring=scoring_meth)).mean()
scores_ub_dum

Unfortunately Dummy model fails to predict any example from the positive class and returns 0 for recall, precission and f1 scores.

In [ ]:
scores_ub_knn=pd.DataFrame(cross_validate(pipe_ub_knn,X_train,y_train,cv=5,return_train_score=True,scoring=scoring_meth)).mean()
scores_ub_knn

The train and validation accuracy for KNN are 0.8634 and 0.8103 respectively. However, accuracy is not the best metrics for assessing a model and other scores can present a better picture.

In [ ]:
scores_ub_svc=pd.DataFrame(cross_validate(pipe_ub_svc,X_train,y_train,cv=5,return_train_score=True,scoring=scoring_meth)).mean()
scores_ub_svc

The scores for SVM are slightly lower than KNN.

In [ ]:
scores_ub_rfc=pd.DataFrame(cross_validate(pipe_ub_rfc,X_train,y_train,cv=5,return_train_score=True,scoring=scoring_meth)).mean()
scores_ub_rfc

All scores have improved dramatically with RandomForestClassifier. However, the bigger gap between training and test results can be a sign of overfitting. 

Another problem to address is the imbalance in the data. Therefore, a new pipeline with RandomForestClassifier with class_weight="balanced" is introduced and similar workflow is followed. 

In [ ]:
pipe_b_rfc=make_pipeline(preprocessor,RandomForestClassifier(class_weight="balanced",random_state=77))

In [ ]:
scores_b_rfc=pd.DataFrame(cross_validate(pipe_b_rfc,X_train,y_train,cv=5,return_train_score=True,scoring=scoring_meth)).mean()
scores_b_rfc

The changes are not drastic. This can be because of the fact that the imbalance is not huge.


In order to find the best parameters, a RandomizedSearchCV named random_search over a grid of two hyperparameters under the name param_dist is carried out. These hyperparameters are n_estimators and max_depth and change over a range of 10 to 300 and 2 to 20 respectively.

In [ ]:
param_dist = {
    "randomforestclassifier__n_estimators": scipy.stats.randint(low=10, high=300),
    "randomforestclassifier__max_depth": scipy.stats.randint(low=2, high=20),
}

In [ ]:
random_search = RandomizedSearchCV (
    pipe_b_rfc,
    param_dist,
    n_iter=50,
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring = "f1",
    random_state=123,
)

In [ ]:
random_search.fit(X_train, y_train)

Now that random_search is fitted on X_train and y_train, we can extract the results. In the following, best parameters and model, and optimal scores are calculated and reported. These scores can be compared with the amounts obtained with default values for hyperparameters. 

In [ ]:
optimal_parameters = random_search.best_params_
optimal_parameters

In [ ]:
optimal_model = random_search.best_estimator_
optimal_model

What we can atain using score function are the accuracies, which have even dropped. However, other scores can be calculated to show the improvement.

In [ ]:
training_score = random_search.score(X_train,y_train)
training_score

In [ ]:
testing_score = random_search.score(X_test,y_test)
testing_score

In [ ]:
optimal_recal=round(recall_score(y_test,random_search.predict(X_test)),2)
optimal_recal

In [ ]:
optimal_precision=round(precision_score(y_test,random_search.predict(X_test),average='weighted'),2)
optimal_precision

In [ ]:
optimal_f1=round(f1_score(y_test,random_search.predict(X_test),average='weighted'),2)
optimal_f1

All scores, except for the accuracy, have increased considerably. 
Results can be presented more clearly using classification_report and plot_confusion_matrix:

In [ ]:
print(classification_report(y_test,random_search.predict(X_test)))


In [ ]:
cm_plot = plot_confusion_matrix(random_search,X_test,y_test)

according the confusion matrix, 118 Negatives and 58 Positives were correctly recognised and the model has failed to predict 18 Negatives and 15 Positives. Although this is a satisfying result, missing 15 Positives from just 73 is not a good sign. 

In [ ]:
numeric_features = ['MoisturePercent']
categorical_features = ['ManufacturerProvCode','ManufacturingTypeEn','CategoryTypeEn','MilkTypeEn','MilkTreatmentTypeEn']
binary_features = ['Organic']
ordinal_features = []
drop_features = ['FlavourEn','CheeseId','RindTypeEn','CheeseName']
text_features = ['CharacteristicsEn']


In [ ]:
preprocessor_text = make_column_transformer(
    (numeric_transformer,numeric_features),
    (categorical_transformer,categorical_features),
    (binary_transformer,binary_features),
    ("drop",drop_features),
    (text_transformer,text_features))

In [ ]:
pipe_LR=make_pipeline(preprocessor_text,LogisticRegression(max_iter=1000))

In [ ]:
scores_LR=pd.DataFrame(cross_validate(pipe_LR,X_train,y_train,cv=5,return_train_score=True,scoring=scoring_meth)).mean()
scores_LR

In [ ]:
param_grid = {
    "logisticregression__C": loguniform(0.01, 100),
    "countvectorizer__max_features": randint(1, 10),
}

In [ ]:
random_search_text=RandomizedSearchCV(pipe_LR,param_grid,n_iter=10,cv=5,random_state=888,n_jobs=-1,verbose=3,return_train_score=True)


In [ ]:
random_search_text.fit(X_train,y_train)

## Discussion

As seen earlier, hyperparameter optimizing improves all scores but accuracy. Actually the data is not balanced so accuracy based optimization leads to a worse prediction. Hence, other methods are of importance. 

In this report, text features are not studied. Obviously they can contribute to making better predictions.

Another interesting question is finding major patterns such as the distribution of FatLevel in different MilkTypeEn subsets for example. 

## Refrences 

Kaggle 

Open Government Licence (Canada)